In [1]:
%jsroot on

In [2]:
#include<TFile.h>
#include<TH1.h>
#include<TLine.h>
#include<TPoint.h>
#include<iostream>
#include<string>
#include<stdio.h>

In [3]:
TFile *f=new TFile("//Analysis/Output_HZZAnalysis/data.root");
//TFile *f = new TFile("//Analysis/Output_HZZAnalysis/ggH125_ZZ4lep.root"); // Prueba MC gg->H125->ZZ->4lep.
mass_four_lep->Print(); // Test de lectura de datos

TH1.Print Name  = mass_four_lep, Entries= 321, Total sum= 33


In [4]:
// gROOT->SetStyle("ATLAS"); // Establecer estilo.
gStyle->SetOptStat("emr"); // Dibujar estadisticas de grafico.
gStyle->SetOptFit(1112);

In [5]:
TCanvas *c1 = new TCanvas("c1","c1",10,10,900,600);
c1->SetGrid();

In [6]:
TH1F *m_four_lep = (TH1F*)f->Get("mass_four_lep");
//m_four_lep->Draw();
m_four_lep->Draw("E1"); // E1: Para representar los puntos con barras de error.

In [7]:
Int_t fit = 5; // Opciones 0, 1, 2, 3, 4 = Distribuciones

In [8]:
Double_t myBW(Double_t* x, Double_t* par) // Distribución de Breit-Wigner relativista
{
    Double_t arg1 = sqrt(pow(par[2],2)*(pow(par[2],2)+pow(par[1],2))); // Gamma=par[1] M=par[2]
    Double_t arg2 = 2*sqrt(2)*par[2]*par[1]*arg1/(TMath::Pi()*sqrt(pow(par[2],2)+arg1));
    Double_t arg3 = pow((pow(x[0],2)-pow(par[2],2)),2);
    Double_t arg4 = pow(par[2],2)*pow(par[1],2);
    return par[0]*arg2/(arg3+arg4);
}

In [9]:
Double_t myCauchy(Double_t* x, Double_t* par)
{   // par[0] = constant, par[1] = x0, par[2]=gamma
    Double_t arg1 = (x[0]-par[1])/par[2];
    Double_t arg2 = 1+pow(arg1,2);
    return par[0]/(TMath::Pi()*par[2]*arg2);
}

In [10]:
Double_t myCauchyDist(Double_t* x, Double_t* par)
{   // Double_t x, Double_t t, Double_t s
    // t is the location parameter
    // s is the scale parameter
    return par[0]*TMath::CauchyDist(x[0],par[1],par[2]);
}

In [12]:
Double_t myBWDist(Double_t* x, Double_t* par)
{   // Double_t x, Double_t mean, Double_t gamma
    return par[0]*TMath::BreitWigner(x[0], par[1], par[2]);
}

In [13]:
if (fit == 0) {
    TF1 *f1 = new TF1("f1","myBW",110,140,3);
    f1->SetParameters(1,m_four_lep->GetRMS(),m_four_lep->GetMean());
    f1->SetParNames ("Constant","Root mean square","Mean"); // OJO con el tipo de parámetro.
}
if (fit == 1) {TF1 *f1 = new TF1("f1","gaus",110,140);}
if (fit == 2) {TF1 *f1 = new TF1("f1","landau",110,140);}
if (fit == 4) {
    TF1 *f1 = new TF1("f1","myCauchy",110,140,3);
    f1->SetParameters(1,m_four_lep->GetMean(),m_four_lep->GetRMS());
    f1->SetParNames ("Constant","Root mean square","Mean"); // OJO con el tipo de parámetro.
}
if (fit == 5) {
    TF1 *f1 = new TF1("f1","myCauchyDist",110,140,3);
    f1->SetParameters(1,m_four_lep->GetMean(),m_four_lep->GetRMS());
    f1->SetParNames ("Constant","Mean","Root mean square"); // OJO con el tipo de parámetro.
}
if (fit == 6) {
    TF1 *f1 = new TF1("f1","myBWDist",110,140,3);
    f1->SetParameters(1,m_four_lep->GetMean(),m_four_lep->GetRMS());
    f1->SetParNames ("Constant","Mean","Root mean square"); // OJO con el tipo de parámetro.
}

In [14]:
m_four_lep->Fit("f1","R"); // Puede usarse V para agregar más detalles estadísticos.
f1->SetLineColor(kRed);

 FCN=29.9282 FROM HESSE     STATUS=FAILED         15 CALLS         182 TOTAL
                     EDM=5.23919e-08    STRATEGY= 1  ERROR MATRIX UNCERTAINTY 100.0 per cent
  EXT PARAMETER                APPROXIMATE        STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Mean         1.24167e+02   3.99671e-01   1.08644e-03   2.21224e-04
   2  Root mean square   1.07419e-01   6.35723e-02   1.71770e-04   7.64214e-03
   3  p2          -1.57298e+05   4.24264e-01  -0.00000e+00   0.00000e+00


In [15]:
// Print the full information of the fit including covariance matrix.
f1->Print("V"); 

// Store the result in a ROOT file.
//f1->Write(); 

Formula based function:     f1 
            tf1lambda : myCauchyDist Ndim= 1, Npar= 3, Number= 0 
 Formula expression: 
	myCauchyDist 
List of  Parameters: 
Par   0                 Mean =  124.166736 
Par   1     Root mean square =    0.107419 
Par   2                   p2 =  -157297.587427 
Expression passed to Cling:
	myCauchyDist


In [16]:
minx = f1->GetXmin(); // Mínimo en x del ajuste.
maxx = f1->GetXmax(); // Máximo en x del ajuste.

mode_x = f1->GetMaximumX(minx,maxx); // Coordenada x del máximo en el ajuste dentro del intervalo [minx,maxx].
mode_y = f1->Eval(mode_x); // Coordenada y del máximo.

upper_halfwidth = f1->GetX(mode_y/2.0,mode_x,maxx);
lower_halfwidth = f1->GetX(mode_y/2.0,minx,mode_x);

width = upper_halfwidth-lower_halfwidth;

cout    << "Coordenada del punto máximo en el ajuste: (" << mode_x << "," << mode_y << ")." << endl
        << "Intersecciones en m4l a média altura: " << lower_halfwidth << " GeV" << " y " << upper_halfwidth << " GeV." << endl
        << "Ancho de decaimiento, \u0393 = " << width << " GeV." << endl
        << "Tiempo de vida media, \u03C4 = " << 1/(width*1E9) << " s." << endl;
        //<< "Tiempo de vida media, \u03C4 = " << 4.135667696E-15/(2*TMath::Pi()*width*1E9) << " s." << endl; // Consultar.


TLine *l1 = new TLine(lower_halfwidth,mode_y/2.0,upper_halfwidth,mode_y/2.0);
l1->SetLineColor(kGreen);
l1->SetLineStyle(7);
l1->Draw("same");

TLine *l2 = new TLine(mode_x,mode_y,mode_x,0);
l2->SetLineColor(kGreen);
l2->SetLineStyle(7);
l2->Draw("same");

c1->Draw();

Coordenada del punto máximo en el ajuste: (124.167,2.96326).
Intersecciones en m4l a média altura: 124.059 GeV y 124.274 GeV.
Ancho de decaimiento, Γ = 0.214838 GeV.
Tiempo de vida media, τ = 4.65467e-09 s.


#### OJO Revisar los orden de magnitud y unidades.